This is the code document for INFOSYS 722 Iteration 4 from Xuan Luo

In [1]:
import findspark
findspark.init('/home/ubuntu/spark-3.2.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('basics').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/09 09:09:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
#read data and show
file1="Cardiovascular Diseases Risk Prediction Dataset dirty - Part 1.csv"
file2="Cardiovascular Diseases Risk Prediction Dataset dirty - Part 2.csv"
# read data
data1 = spark.read.csv(file1, header=True, inferSchema=True)
data2 = spark.read.csv(file2, header=True, inferSchema=True)

# show and print data
data1.show()
data2.show()

+--------------+--------------------+--------+------------+----------+-----------+----------+--------+---------+------+-----------+----------+----------+-----+--------------+------------------+----------------+--------------------------+----------------------+
|General_Health|             Checkup|Exercise|HeartDisease|SkinCancer|OtherCancer|Depression|Diabetes|Arthritis|   Sex|AgeCategory|Height(cm)|Weight(kg)|  BMI|SmokingHistory|AlcoholConsumption|FruitConsumption|GreenVegetablesConsumption|FriedPotatoConsumption|
+--------------+--------------------+--------+------------+----------+-----------+----------+--------+---------+------+-----------+----------+----------+-----+--------------+------------------+----------------+--------------------------+----------------------+
|          Poor|Within the past 2...|      No|          No|        No|         No|        No|      No|      Yes|Female|      70-74|       150|     32.66|14.54|           Yes|                 0|              30|       

In [3]:
#print data schema
data1.printSchema()
data2.printSchema()

root
 |-- General_Health: string (nullable = true)
 |-- Checkup: string (nullable = true)
 |-- Exercise: string (nullable = true)
 |-- HeartDisease: string (nullable = true)
 |-- SkinCancer: string (nullable = true)
 |-- OtherCancer: string (nullable = true)
 |-- Depression: string (nullable = true)
 |-- Diabetes: string (nullable = true)
 |-- Arthritis: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- AgeCategory: string (nullable = true)
 |-- Height(cm): integer (nullable = true)
 |-- Weight(kg): double (nullable = true)
 |-- BMI: double (nullable = true)
 |-- SmokingHistory: string (nullable = true)
 |-- AlcoholConsumption: integer (nullable = true)
 |-- FruitConsumption: integer (nullable = true)
 |-- GreenVegetablesConsumption: integer (nullable = true)
 |-- FriedPotatoConsumption: integer (nullable = true)

root
 |-- General_Health: string (nullable = true)
 |-- Checkup: string (nullable = true)
 |-- Exercise: string (nullable = true)
 |-- HeartDisease: string (nu

In [4]:
# Define numerical features and categorical features
numerical_features = ['Height(cm)', 'Weight(kg)', 'BMI', 'AlcoholConsumption',
                      'FruitConsumption', 'GreenVegetablesConsumption', 
                      'FriedPotatoConsumption']
categorical_features = ['General_Health','Checkup', 'Exercise', 'HeartDisease',
                        'SkinCancer', 'OtherCancer', 'Depression', 'Diabetes', 
                        'Arthritis', 'Sex', 'AgeCategory', 'SmokingHistory']

In [5]:
'''Explore the data'''
# Numerical feature explore
data1.select(numerical_features).describe().show()
data2.select(numerical_features).describe().show()

+-------+------------------+-----------------+------------------+------------------+------------------+--------------------------+----------------------+
|summary|        Height(cm)|       Weight(kg)|               BMI|AlcoholConsumption|  FruitConsumption|GreenVegetablesConsumption|FriedPotatoConsumption|
+-------+------------------+-----------------+------------------+------------------+------------------+--------------------------+----------------------+
|  count|            151469|           151468|            151471|            151471|            151470|                    151470|                151468|
|   mean|170.72150076913428|83.52362604643292| 28.56556799651395| 5.199133827597362|29.813065293457452|         14.99376774278735|     6.301093300235033|
| stddev|10.620184100377434|21.41840277904633|6.5344630930037075| 8.267109622728807|24.798504144686706|        14.810641104656465|      8.55474523326812|
|    min|                91|            24.95|             12.11|           

In [6]:
# statistical explore
from pyspark.sql import functions as F

stats = ["avg", "stddev", "skewness", "kurtosis", "variance"]

for stat in stats:
    agg_expr = [getattr(F, stat)(col_name).alias(f"{col_name}_{stat}") for col_name in numerical_features]
    data1.agg(*agg_expr).show()

+------------------+-----------------+-----------------+----------------------+--------------------+------------------------------+--------------------------+
|    Height(cm)_avg|   Weight(kg)_avg|          BMI_avg|AlcoholConsumption_avg|FruitConsumption_avg|GreenVegetablesConsumption_avg|FriedPotatoConsumption_avg|
+------------------+-----------------+-----------------+----------------------+--------------------+------------------------------+--------------------------+
|170.72150076913428|83.52362604643292|28.56556799651395|     5.199133827597362|  29.813065293457452|             14.99376774278735|         6.301093300235033|
+------------------+-----------------+-----------------+----------------------+--------------------+------------------------------+--------------------------+



+------------------+-----------------+------------------+-------------------------+-----------------------+---------------------------------+-----------------------------+
| Height(cm)_stddev|Weight(kg)_stddev|        BMI_stddev|AlcoholConsumption_stddev|FruitConsumption_stddev|GreenVegetablesConsumption_stddev|FriedPotatoConsumption_stddev|
+------------------+-----------------+------------------+-------------------------+-----------------------+---------------------------------+-----------------------------+
|10.620184100377434|21.41840277904633|6.5344630930037075|        8.267109622728807|     24.798504144686706|               14.810641104656465|             8.55474523326812|
+------------------+-----------------+------------------+-------------------------+-----------------------+---------------------------------+-----------------------------+



23/10/09 09:10:14 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+-------------------+------------------+---------------------------+-------------------------+-----------------------------------+-------------------------------+
| Height(cm)_skewness|Weight(kg)_skewness|      BMI_skewness|AlcoholConsumption_skewness|FruitConsumption_skewness|GreenVegetablesConsumption_skewness|FriedPotatoConsumption_skewness|
+--------------------+-------------------+------------------+---------------------------+-------------------------+-----------------------------------+-------------------------------+
|0.024511764298717056| 1.0644044154816517|1.3747440104203876|         1.8499343604195304|       1.2470255656973934|                 2.4076690955367046|              4.926845407866809|
+--------------------+-------------------+------------------+---------------------------+-------------------------+-----------------------------------+-------------------------------+

+--------------------+-------------------+-----------------+-------------------

In [7]:
from pyspark.sql.functions import when
 
# transfer yes to 1 and no to 0 in HeartDisease column
data11 = data1.withColumn("HeartDisease", when(data1["HeartDisease"] == "yes", 1).otherwise(0))
data22 = data2.withColumn("HeartDisease", when(data2["HeartDisease"] == "yes", 1).otherwise(0))

# print schma to make sure that the transfer is successfull
data11.printSchema()
data22.printSchema()

root
 |-- General_Health: string (nullable = true)
 |-- Checkup: string (nullable = true)
 |-- Exercise: string (nullable = true)
 |-- HeartDisease: integer (nullable = false)
 |-- SkinCancer: string (nullable = true)
 |-- OtherCancer: string (nullable = true)
 |-- Depression: string (nullable = true)
 |-- Diabetes: string (nullable = true)
 |-- Arthritis: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- AgeCategory: string (nullable = true)
 |-- Height(cm): integer (nullable = true)
 |-- Weight(kg): double (nullable = true)
 |-- BMI: double (nullable = true)
 |-- SmokingHistory: string (nullable = true)
 |-- AlcoholConsumption: integer (nullable = true)
 |-- FruitConsumption: integer (nullable = true)
 |-- GreenVegetablesConsumption: integer (nullable = true)
 |-- FriedPotatoConsumption: integer (nullable = true)

root
 |-- General_Health: string (nullable = true)
 |-- Checkup: string (nullable = true)
 |-- Exercise: string (nullable = true)
 |-- HeartDisease: integer 

In [8]:
# Correlation exploration
for feature in numerical_features:
    correlation = data11.corr(feature, "HeartDisease")
    print(f"Correlation between {feature} and HeartDisease: {correlation}")
    
print("-------------------------------------------")

for feature in numerical_features:
    correlation = data22.corr(feature, "HeartDisease")
    print(f"Correlation between {feature} and HeartDisease: {correlation}")

Correlation between Height(cm) and HeartDisease: nan
Correlation between Weight(kg) and HeartDisease: nan
Correlation between BMI and HeartDisease: nan
Correlation between AlcoholConsumption and HeartDisease: nan
Correlation between FruitConsumption and HeartDisease: nan
Correlation between GreenVegetablesConsumption and HeartDisease: nan
Correlation between FriedPotatoConsumption and HeartDisease: nan
-------------------------------------------
Correlation between Height(cm) and HeartDisease: nan
Correlation between Weight(kg) and HeartDisease: nan
Correlation between BMI and HeartDisease: nan
Correlation between AlcoholConsumption and HeartDisease: nan
Correlation between FruitConsumption and HeartDisease: nan
Correlation between GreenVegetablesConsumption and HeartDisease: nan
Correlation between FriedPotatoConsumption and HeartDisease: nan


In [9]:
# Categorical feature explore  
data1.groupBy('HeartDisease').count().show()  # Exploring target variable
data2.groupBy('HeartDisease').count().show()  # Exploring target variable

for feature in categorical_features:
    print(f"Counts for {feature} in data1:")
    data1.groupBy(feature).count().orderBy('count', ascending=False).show()
    print(f"Counts for {feature} in data2:")
    data2.groupBy(feature).count().orderBy('count', ascending=False).show()

+------------+------+
|HeartDisease| count|
+------------+------+
|          No|139401|
|         Yes| 12070|
+------------+------+

+------------+------+
|HeartDisease| count|
+------------+------+
|          No|144482|
|         Yes| 12901|
+------------+------+

Counts for General_Health in data1:
+--------------+-----+
|General_Health|count|
+--------------+-----+
|     Very Good|54361|
|          Good|46682|
|     Excellent|27521|
|          Fair|17350|
|          Poor| 5557|
+--------------+-----+

Counts for General_Health in data2:
+--------------+-----+
|General_Health|count|
+--------------+-----+
|     Very Good|56034|
|          Good|48682|
|     Excellent|28433|
|          Fair|18460|
|          Poor| 5774|
+--------------+-----+

Counts for Checkup in data1:
+--------------------+------+
|             Checkup| count|
+--------------------+------+
|Within the past year|116765|
|Within the past 2...| 18700|
|Within the past 5...|  8686|
| 5 or more years ago|  6619|
|      

In [10]:
from pyspark.sql.functions import col, sum as pyspark_sum

# Checking for missing values in data1
print("Missing values in data1:")
data1.select(*(pyspark_sum(col(c).isNull().cast("int")).alias(c) for c in data1.columns)).show()

# Checking for missing values in data2
print("Missing values in data2:")
data2.select(*(pyspark_sum(col(c).isNull().cast("int")).alias(c) for c in data2.columns)).show()

Missing values in data1:


+--------------+-------+--------+------------+----------+-----------+----------+--------+---------+---+-----------+----------+----------+---+--------------+------------------+----------------+--------------------------+----------------------+
|General_Health|Checkup|Exercise|HeartDisease|SkinCancer|OtherCancer|Depression|Diabetes|Arthritis|Sex|AgeCategory|Height(cm)|Weight(kg)|BMI|SmokingHistory|AlcoholConsumption|FruitConsumption|GreenVegetablesConsumption|FriedPotatoConsumption|
+--------------+-------+--------+------------+----------+-----------+----------+--------+---------+---+-----------+----------+----------+---+--------------+------------------+----------------+--------------------------+----------------------+
|             0|      1|       0|           0|         0|          2|         2|       1|        3|  0|          1|         2|         3|  0|             2|                 0|               1|                         1|                     3|
+--------------+-------+----

In [11]:
from pyspark.sql.functions import col, percentile_approx

for feature in numerical_features:
    # Calculate Q1 and Q3
    quantiles = data1.approxQuantile(feature, [0.25, 0.75], 0.01)
    Q1 = quantiles[0]
    Q3 = quantiles[1]
    # Compute IQR
    IQR = Q3 - Q1  
    # Define boundaries for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    # Filter out outliers
    outliers = data1.filter((col(feature) < lower_bound) | (col(feature) > upper_bound))
    # Print results
    print(f"Outliers for {feature}: {outliers.count()}")

Outliers for Height(cm): 492
Outliers for Weight(kg): 3607
Outliers for BMI: 5198
Outliers for AlcoholConsumption: 18141
Outliers for FruitConsumption: 33484
Outliers for GreenVegetablesConsumption: 5756
Outliers for FriedPotatoConsumption: 9570


In [12]:
duplicates_data1 = data1.groupBy(data1.columns).count().filter("`count` > 1")
num_duplicates_data1 = duplicates_data1.count()
print(f"Number of duplicate rows in data1: {num_duplicates_data1}")

duplicates_data2 = data1.groupBy(data2.columns).count().filter("`count` > 1")
num_duplicates_data2 = duplicates_data2.count()
print(f"Number of duplicate rows in data2: {num_duplicates_data2}")

Number of duplicate rows in data1: 24


Number of duplicate rows in data2: 24


In [13]:
# Function to compute desired statistics for a given column
def compute_stats(df, col_name):
    basic_stats = df.agg(
        F.count(F.col(col_name)).alias('count'),
        F.mean(F.col(col_name)).alias('mean'),
        F.stddev(F.col(col_name)).alias('stddev'),
        F.min(F.col(col_name)).alias('min'),
        F.max(F.col(col_name)).alias('max')
    ).collect()[0]
    quantiles = df.approxQuantile(col_name, [0.25, 0.5, 0.75], 0.01)   
    return (
        basic_stats['count'],
        basic_stats['mean'],
        basic_stats['stddev'],
        basic_stats['min'],
        quantiles[0],
        quantiles[1],
        quantiles[2],
        basic_stats['max']
    )
# Display statistics for each column in numerical_features
for col_name in numerical_features:
    print(f"Statistics for {col_name} in data1:")
    stats_data1 = compute_stats(data1, col_name)
    print(f"Count: {stats_data1[0]}, Mean: {stats_data1[1]:.2f}, Stddev: {stats_data1[2]:.2f},\
          Min: {stats_data1[3]}, 25th percentile: {stats_data1[4]:.2f}, Median: {stats_data1[5]:.2f},\
          75th percentile: {stats_data1[6]:.2f}, Max: {stats_data1[7]}")    
    print(f"Statistics for {col_name} in data2:")
    stats_data2 = compute_stats(data2, col_name)
    print(f"Count: {stats_data2[0]}, Mean: {stats_data2[1]:.2f}, Stddev: {stats_data2[2]:.2f},\
          Min: {stats_data2[3]}, 25th percentile: {stats_data2[4]:.2f},\
          Median: {stats_data2[5]:.2f}, 75th percentile: {stats_data2[6]:.2f}, Max: {stats_data2[7]}")   
    print("-"*50)

Statistics for Height(cm) in data1:
Count: 151469, Mean: 170.72, Stddev: 10.62,          Min: 91, 25th percentile: 163.00, Median: 170.00,          75th percentile: 178.00, Max: 234
Statistics for Height(cm) in data2:
Count: 157383, Mean: 170.51, Stddev: 10.69,          Min: 91, 25th percentile: 163.00,          Median: 170.00, 75th percentile: 178.00, Max: 241
--------------------------------------------------
Statistics for Weight(kg) in data1:
Count: 151468, Mean: 83.52, Stddev: 21.42,          Min: 24.95, 25th percentile: 68.04, Median: 81.19,          75th percentile: 95.25, Max: 293.02
Statistics for Weight(kg) in data2:
Count: 157383, Mean: 83.65, Stddev: 21.27,          Min: 26.31, 25th percentile: 68.04,          Median: 81.65, 75th percentile: 95.25, Max: 283.5
--------------------------------------------------
Statistics for BMI in data1:
Count: 151471, Mean: 28.57, Stddev: 6.53,          Min: 12.11, 25th percentile: 24.14, Median: 27.41,          75th percentile: 31.62, Max

In [14]:
#Drop the field that is irrelevant to business objectives
data1 = data1.drop('Checkup')
data2 = data2.drop('Checkup')
data1.printSchema()
data2.printSchema()

root
 |-- General_Health: string (nullable = true)
 |-- Exercise: string (nullable = true)
 |-- HeartDisease: string (nullable = true)
 |-- SkinCancer: string (nullable = true)
 |-- OtherCancer: string (nullable = true)
 |-- Depression: string (nullable = true)
 |-- Diabetes: string (nullable = true)
 |-- Arthritis: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- AgeCategory: string (nullable = true)
 |-- Height(cm): integer (nullable = true)
 |-- Weight(kg): double (nullable = true)
 |-- BMI: double (nullable = true)
 |-- SmokingHistory: string (nullable = true)
 |-- AlcoholConsumption: integer (nullable = true)
 |-- FruitConsumption: integer (nullable = true)
 |-- GreenVegetablesConsumption: integer (nullable = true)
 |-- FriedPotatoConsumption: integer (nullable = true)

root
 |-- General_Health: string (nullable = true)
 |-- Exercise: string (nullable = true)
 |-- HeartDisease: string (nullable = true)
 |-- SkinCancer: string (nullable = true)
 |-- OtherCancer: str

In [15]:
#Remove null value
# Removing rows with null or missing values from data1
data1= data1.na.drop()

# Removing rows with null or missing values from data2
data2 = data2.na.drop()

In [16]:
#check remove missing value is successful or not
# Checking for missing values in data1
print("Missing values in data1:")
data1.select(*(pyspark_sum(col(c).isNull().cast("int")).alias(c) for c in data1.columns)).show()

# Checking for missing values in data2
print("Missing values in data2:")
data2.select(*(pyspark_sum(col(c).isNull().cast("int")).alias(c) for c in data2.columns)).show()

Missing values in data1:


+--------------+--------+------------+----------+-----------+----------+--------+---------+---+-----------+----------+----------+---+--------------+------------------+----------------+--------------------------+----------------------+
|General_Health|Exercise|HeartDisease|SkinCancer|OtherCancer|Depression|Diabetes|Arthritis|Sex|AgeCategory|Height(cm)|Weight(kg)|BMI|SmokingHistory|AlcoholConsumption|FruitConsumption|GreenVegetablesConsumption|FriedPotatoConsumption|
+--------------+--------+------------+----------+-----------+----------+--------+---------+---+-----------+----------+----------+---+--------------+------------------+----------------+--------------------------+----------------------+
|             0|       0|           0|         0|          0|         0|       0|        0|  0|          0|         0|         0|  0|             0|                 0|               0|                         0|                     0|
+--------------+--------+------------+----------+-----------

In [17]:
#show AgeCategory count before remove unclassify value
print("Before remove unclassify value")
data1.groupBy('AgeCategory').count().show()
#remove unclassify value in Agecategories
data1 = data1.filter(data1['AgeCategory'] != '4')
#show AgeCategory count after remove unclassify value
print("After remove unclassify value")
data1.groupBy('AgeCategory').count().show()

Before remove unclassify value


+-----------+-----+
|AgeCategory|count|
+-----------+-----+
|      65-69|16062|
|      75-79|10041|
|      30-34| 9184|
|        80+|10662|
|      70-74|15133|
|      18-24| 9442|
|          4|    1|
|      50-54|12384|
|      35-39|10310|
|      45-49|10269|
|      25-29| 7706|
|      40-44|10666|
|      55-59|13803|
|      60-64|15787|
+-----------+-----+

After remove unclassify value


+-----------+-----+
|AgeCategory|count|
+-----------+-----+
|      65-69|16062|
|      75-79|10041|
|      30-34| 9184|
|        80+|10662|
|      70-74|15133|
|      18-24| 9442|
|      50-54|12384|
|      35-39|10310|
|      45-49|10269|
|      25-29| 7706|
|      40-44|10666|
|      55-59|13803|
|      60-64|15787|
+-----------+-----+



In [18]:
#show Diabetes count before remove unclassify value
print("Before remove unclassify value")
data1.groupBy('Diabetes').count().show()
data2.groupBy('Diabetes').count().show()
#remove unclassify value in Diabetes
data1 = data1.withColumn(
    'Diabetes',
    when(data1['Diabetes'] == 'No, pre-diabetes or borderline diabetes', 'No')
    .when(data1['Diabetes'] == 'Yes, but female told only during pregnancy', 'Yes')
    .otherwise(data1['Diabetes'])
)
data2 = data2.withColumn(
    'Diabetes',
    when(data2['Diabetes'] == 'No, pre-diabetes or borderline diabetes', 'No')
    .when(data2['Diabetes'] == 'Yes, but female told only during pregnancy', 'Yes')
    .otherwise(data2['Diabetes'])
)
#show Diabetes count after remove unclassify value
print("After remove unclassify value")
data1.groupBy('Diabetes').count().show()
data2.groupBy('Diabetes').count().show()

Before remove unclassify value


+--------------------+------+
|            Diabetes| count|
+--------------------+------+
|No, pre-diabetes ...|  3488|
|Yes, but female t...|  1285|
|                  No|127385|
|                 Yes| 19291|
+--------------------+------+

+--------------------+------+
|            Diabetes| count|
+--------------------+------+
|No, pre-diabetes ...|  3408|
|Yes, but female t...|  1361|
|                  No|131738|
|                 Yes| 20876|
+--------------------+------+

After remove unclassify value
+--------+------+
|Diabetes| count|
+--------+------+
|      No|130873|
|     Yes| 20576|
+--------+------+

+--------+------+
|Diabetes| count|
+--------+------+
|      No|135146|
|     Yes| 22237|
+--------+------+



In [19]:
from pyspark.sql.functions import col

# Count duplicates
duplicate_count_data1 = data1.groupBy(data1.columns).count().filter(col('count') > 1).count()
duplicate_count_data2 = data2.groupBy(data2.columns).count().filter(col('count') > 1).count()
duplicate_count = duplicate_count_data1 + duplicate_count_data2
# Print the count
print("Number of duplicates before remove:", duplicate_count)

# Remove duplicate rows
data1 = data1.dropDuplicates()
data2 = data2.dropDuplicates()

# Count duplicates after removal
duplicate_count_data1_after = data1.groupBy(data1.columns).count().filter(col('count') > 1).count()
duplicate_count_data2_after = data2.groupBy(data2.columns).count().filter(col('count') > 1).count()
duplicate_count_after = duplicate_count_data1_after + duplicate_count_data2_after

# Print the count
print("Number of duplicates after remove:", duplicate_count_after)

Number of duplicates before remove: 74


Number of duplicates after remove: 0


In [20]:
#append data1 and data2
df = data1.union(data2)
for feature in numerical_features:
    # Calculate Q1 and Q3
    quantiles = df.approxQuantile(feature, [0.25, 0.75], 0.01)
    Q1 = quantiles[0]
    Q3 = quantiles[1]
    # Compute IQR
    IQR = Q3 - Q1  
    # Define boundaries for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    # Filter out outliers
    outliers = df.filter((col(feature) < lower_bound) | (col(feature) > upper_bound))
    # Print results
    print(f"Outliers for {feature} before remove outlier: {outliers.count()}")
    # Removing outliers from data1
    df = df.filter((col(feature) >= lower_bound) & (col(feature) <= upper_bound))
    # Filter out outliers
    outliers = df.filter((col(feature) < lower_bound) | (col(feature) > upper_bound))
    # Print results
    print(f"Outliers for {feature} after remove outlier: {outliers.count()}")

Outliers for Height(cm) before remove outlier: 1055
Outliers for Height(cm) after remove outlier: 0


Outliers for Weight(kg) before remove outlier: 7205
Outliers for Weight(kg) after remove outlier: 0


Outliers for BMI before remove outlier: 6453


Outliers for BMI after remove outlier: 0


Outliers for AlcoholConsumption before remove outlier: 35406


Outliers for AlcoholConsumption after remove outlier: 0


Outliers for FruitConsumption before remove outlier: 58266


Outliers for FruitConsumption after remove outlier: 0


Outliers for GreenVegetablesConsumption before remove outlier: 5246
Outliers for GreenVegetablesConsumption after remove outlier: 0


Outliers for FriedPotatoConsumption before remove outlier: 12375
Outliers for FriedPotatoConsumption after remove outlier: 0


In [21]:
#show AlcoholConsumption count
df.groupBy('AlcoholConsumption').count().show()

+------------------+-----+
|AlcoholConsumption|count|
+------------------+-----+
|                12| 5900|
|                 1|16711|
|                13|   53|
|                 6| 2423|
|                 3| 7748|
|                 5| 6654|
|                15| 4355|
|                 9|  162|
|                 4|15776|
|                 8| 8958|
|                 7| 1822|
|                10| 5449|
|                11|   26|
|                14|  338|
|                 2|13288|
|                 0|93089|
+------------------+-----+



In [22]:
#print data schma before construct data
df.printSchema()
#construct data
df = df.withColumn("DrinkingAlcohol", when(col("AlcoholConsumption") == 0, "No").otherwise("Yes"))
#print data schma after construct data
df.printSchema()

root
 |-- General_Health: string (nullable = true)
 |-- Exercise: string (nullable = true)
 |-- HeartDisease: string (nullable = true)
 |-- SkinCancer: string (nullable = true)
 |-- OtherCancer: string (nullable = true)
 |-- Depression: string (nullable = true)
 |-- Diabetes: string (nullable = true)
 |-- Arthritis: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- AgeCategory: string (nullable = true)
 |-- Height(cm): integer (nullable = true)
 |-- Weight(kg): double (nullable = true)
 |-- BMI: double (nullable = true)
 |-- SmokingHistory: string (nullable = true)
 |-- AlcoholConsumption: integer (nullable = true)
 |-- FruitConsumption: integer (nullable = true)
 |-- GreenVegetablesConsumption: integer (nullable = true)
 |-- FriedPotatoConsumption: integer (nullable = true)

root
 |-- General_Health: string (nullable = true)
 |-- Exercise: string (nullable = true)
 |-- HeartDisease: string (nullable = true)
 |-- SkinCancer: string (nullable = true)
 |-- OtherCancer: str